In [1]:
import sys
import soundcloud

In [2]:
from functools import partial
from utils import get_results, handle_http_errors

In [3]:
def getAttr(resource, attr):
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [4]:
# A global artist graph used to iterate through the various algorithms.
# Each node is artist id, with edges weighted by activity between then.

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')
name = input("Enter a soundcloud artist to analyze: ")

Enter a soundcloud artist to analyze: Sybyr


In [5]:
# Artist of interest

search = client.get('/users/', q = name)[0]

In [6]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/07/23 00:33:51 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 436,
  'public_favorites_count': 0,
  'followers_count': 51877,
  'followings_count': 22,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 102,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count':

In [7]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [8]:
id2username_cache = {}

# Creating Decorator: handle_http_errors

In [9]:
@handle_http_errors
def id2username(uid, kind='users'):
    global id2username_dict
    username = id2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    uid_str = str(uid)
    get = '/{}/{}'.format(kind, uid_str)
    result = client.get(get)
    
    if kind in ['tracks','comments']:
        username = result.user['username']
    else:
        username = result.username
    # encode it correctly
    # username = str(username.encode('utf-8'))
    # print(username)
    id2username_cache[uid] = username
    
    return username

# 1. Recoding Basic Functions

In [10]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    track_comments = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return track_comments

@handle_http_errors
def getTrackFavoriters(track):
    track_favoriters = get_results(client, '/tracks/{0:s}/favoriters/'.format(str(track)))
    return track_favoriters

# 1b. Testing Recoded Functions

In [11]:
# list of profiles to query
profiles_to_query = [search.id]
profiles_to_query

[13377746]

In [12]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

Only uncomment the code in the loop if you are willing to wait a while to run the cell below:

In [13]:
for t in range(depth):
    print("Iteration " + str(t))
    profiles_to_query = list(set(profiles_to_query))
    
    for profile in profiles_to_query:
        username = id2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))

            encountered_follows = getFollows(profile)
            print("encountered follows: " + ", ".join([getUsername(user) if isinstance(getUsername(user), str) else str(getid(user)) for user in encountered_follows]))
            print('\n')

            encountered_user_favorites = getUserFavorites(profile)
            print("encountered user favorites: " + ", ".join([getUsername(track) if isinstance(getUsername(track), str) else str(id2username(track.id,'tracks')) for track in encountered_user_favorites]))
            print('\n')

            encountered_user_comments = getUserComments(profile)
            print("encountered user comments: " + ", ".join([getUsername(comment) if isinstance(getUsername(comment), str) else str(id2username(comment.track_id,'tracks')) for comment in encountered_user_comments]))#([str(comment.__dict__) for comment in encountered_comments]))#
            print('\n')

#             encountered_followers = getFollowers(profile)
#             print("encountered followers: " + ", ".join([getUsername(user) if isinstance(getUsername(user), str) else str(getid(user)) for user in encountered_followers]))
#             print('\n')

#             encountered_tracks = getTracks(profile)

#             encountered_track_comments = []
#             encountered_track_favoriters = []

#             for track in encountered_tracks:
#                 print("getting comments / favoriters from user track, {}".format(track.title))
#                 encountered_track_comments += getTrackComments(track.id)
#                 encountered_track_favoriters += getTrackFavoriters(track.id)

#             print(encountered_track_comments[0].__dict__)
#             print(encountered_track_favoriters[0].__dict__)

#             print("encountered track comments: " + ", ".join([getUsername(comment) if isinstance(getUsername(comment), str) else comment.user['username'] for comment in encountered_track_comments]))#([str(comment.__dict__) for comment in encountered_track_comments]))#
#             print("encountered track favoriters: " + ", ".join([getUsername(favoriter) if isinstance(getUsername(favoriter), str) else favoriter.user['username'] for favoriter in encountered_track_favoriters]))
        else:
            print("\t", "Artist ID %s is not query-able" % profile)
            unavailable_profiles.append(profile)            

Iteration 0
	 Enqueueing: Sybyr (13377746)
encountered follows: @DevinWillDo, OWINILLSIN {Anti_World}, ERIC NORTH (@RI0T_ANGEL), ANTI-WORLD, MONĀE, neighborhood arion, prod. BABE, JULIEN ANDREAS, Raz Nein, lil xelly (@lilxelly), Landfill, Shark, RIOT_ANGEL, LIL MAI 2020, win32, Ghostie, PE$O EVANS, WIFIGAWD AKA UPT SOULJAH, DariBroko, Leon'sWOLF, CHACHI


encountered user favorites: LIL MAI 2020, TERR9R, ERIC NORTH (@RI0T_ANGEL), Ghostie, lil xelly (@lilxelly), lil xelly (@lilxelly), lil xelly (@lilxelly), Future, BREEZY SUPREME, Carl Wheezer, Leon'sWOLF, ERIC NORTH (@RI0T_ANGEL), AMIR ZERA, Leon'sWOLF, Shark, CHACHI, Leon'sWOLF, BeatPlugg, ERIC NORTH (@RI0T_ANGEL), HStreetRaps, Lildudeluciano, Shark, ANTI-WORLD, Detaro HeiH, LIL MAI 2020, ERIC NORTH (@RI0T_ANGEL), ERIC NORTH (@RI0T_ANGEL), RIOT_ANGEL, YNW BSlime, CXRPSE, CXRPSE, CXRPSE, Helly Is Hansyn, Helly Is Hansyn, NLE Choppa, CXRPSE, DariBroko, Leon'sWOLF, CXRPSE, LIL MAI 2020, ERIC NORTH (@RI0T_ANGEL), Shark, ERIC NORTH (@RI0T_

# 2. Adding even more functions

In [14]:
def getUserInfo(profile):
    info = ['id','permalink', 'username', 'avatar_url', 'country', 'city', 
            'website', 'track_count', 'followers_count', 'followings_count']
    return {i: getAttr(profile, i) for i in info}

def getTrackInfo(track):
    info = ['id', 'user_id', 'created_at', 'streamabale', 'downloadable',
            'playback_count', 'download_count', 'favoritings_count','comment_count']
    return {i: getAttr(track, i) for i in info}

def getUserFavInfo(fav):
    info = ['id', 'user_id','last_modified']
    return {i: getAttr(fav, i) for i in info}

def getTrackFavInfo(fav):
    info = ['id', 'username', 'last_modified']
    return {i: getAttr(fav, i) for i in info}

def getUserCommInfo(comm):
    info = ['id', 'user_id', 'track_id', 'timestamp']
    return {i: getAttr(comm, i) for i in info}

def getTrackCommInfo(comm):
    info = ['id', 'user_id', 'track_id','timestamp']
    return {i: getAttr(comm, i) for i in info}

# Graph Code (In Progress)

Brought over code from print_graph.py for print_graph function. Still need to correctly instantiate nx.MultiDiGraph object with actual data.

In [15]:
# import networkx as nx

# profileGraph = nx.MultiDiGraph()

In [16]:
# def print_graph(G):
#     for artist in G.nodes():
#         if artist:
#             try:
#                 username = id2username(artist)
#                 followings = G.successors(artist)
#                 followers = G.predecessors(artist)
#                 try:    
#                     print("\t", username + " has " + str(len(followings)) + " followings")
#                     print("\t", username + " follows " + ", ".join(map(lambda x: id2username(x), followings)))
#                 except TypeError: 
#                     print("No followings home!")
#                 try:    
#                     print("\t", username + " has " + str(len(followers)) + " followers")
#                     print("\t", username + " is followed by " + ", ".join(map(lambda x: id2username(x), followers)))
#                 except TypeError:
#                     print("No followers home!")
#                     print("-"*40)
#             except UnicodeError:
#                 print("Artist's username not found")

# print("The profile graph currently contains " + str(len(profileGraph.nodes())) + " profiles.")
# print("Here are their connections.")
# print(print_graph(profileGraph))
# print("The profile graph currently contains " + str(nx.number_strongly_connected_components(profileGraph)) + " strongly connected components.")